In [83]:
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom python")
from silvhua import *
import numpy as np
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\projects\midterm-2022-10-24\src")
from midterm_functions import *
import psycopg2

# Flight data

In [84]:
# Retrieve select columns from flights table
query = """
SELECT mkt_carrier,
    mkt_carrier_fl_num,
    fl_date,
    origin_airport_id,
    dest_airport_id,
    crs_dep_time,
    dep_time,
    dep_delay,
    crs_arr_time,
    arr_time,
    arr_delay,
    crs_elapsed_time,
    air_time,
    distance,
    origin_city_name,
    dest_city_name
    FROM flights
WHERE extract(month FROM date(fl_date)) = 1 
AND extract(day FROM date(fl_date)) BETWEEN 1 AND 7
ORDER BY RANDOM()
"""
flights = execute_query(query,limit=1200000)


SELECT mkt_carrier,
    mkt_carrier_fl_num,
    fl_date,
    origin_airport_id,
    dest_airport_id,
    crs_dep_time,
    dep_time,
    dep_delay,
    crs_arr_time,
    arr_time,
    arr_delay,
    crs_elapsed_time,
    air_time,
    distance,
    origin_city_name,
    dest_city_name
    FROM flights
WHERE extract(month FROM date(fl_date)) = 1 
AND extract(day FROM date(fl_date)) BETWEEN 1 AND 7
ORDER BY RANDOM()
 LIMIT 1200000


C:\Users\silvh\OneDrive\lighthouse\projects\midterm-2022-10-24\src\midterm_functions.py:65: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  response = pd.read_sql_query(query_string, con)


In [85]:
flights.shape

(280344, 16)

In [87]:
flights.columns

Index(['mkt_carrier', 'mkt_carrier_fl_num', 'fl_date', 'origin_airport_id',
       'dest_airport_id', 'crs_dep_time', 'dep_time', 'dep_delay',
       'crs_arr_time', 'arr_time', 'arr_delay', 'crs_elapsed_time', 'air_time',
       'distance', 'origin_city_name', 'dest_city_name'],
      dtype='object')

In [88]:
flights['haul_length'] = length  

    # TIME
# Converting time into 24 hours
flights['crs_arr_hrs'] = (flights['crs_arr_time']/100).astype(int)
flights['crs_dep_hrs'] = (flights['crs_dep_time']/100).astype(int)

# Convert time into categories: crs_dep_hrs
ctg = []
for i in flights['crs_dep_hrs']:
    if (i>=5) and (i<12):
        ctg.append('Morning')
    elif (i>=12) and (i<16):
        ctg.append('Afternoon')
    elif (i>=16) and (i<=22):
        ctg.append('Evening')
    elif (i>22) or (i<5):
        ctg.append('Night')

flights['dep_hrs_ctg'] = ctg

# Convert time into categories: crs_arr_hrs
ctg = []
for i in flights['crs_arr_hrs']:
    if (i>=5) and (i<12):
        ctg.append('Morning')
    elif (i>=12) and (i<16):
        ctg.append('Afternoon')
    elif (i>=16) and (i<=22):
        ctg.append('Evening')
    elif (i>22) or (i<5):
        ctg.append('Night')
flights['arr_hrs_ctg'] = ctg


In [89]:
flights.columns

Index(['mkt_carrier', 'mkt_carrier_fl_num', 'fl_date', 'origin_airport_id',
       'dest_airport_id', 'crs_dep_time', 'dep_time', 'dep_delay',
       'crs_arr_time', 'arr_time', 'arr_delay', 'crs_elapsed_time', 'air_time',
       'distance', 'origin_city_name', 'dest_city_name', 'haul_length',
       'crs_arr_hrs', 'crs_dep_hrs', 'dep_hrs_ctg', 'arr_hrs_ctg'],
      dtype='object')

In [90]:
    # CONVERT crs_dep_time TO DATETIME OBJECT
column = 'crs_dep_time'
flights = time_columns(flights,column,format='%H%M',dropna=True)


    # To flights table, add columns with the  features for date and forecasting date
flights = date_forecast_columns(flights,date_column='fl_date',format='%Y-%m-%d')

    # Calculate mean dep_delay and arr_delay for a given carrier 
groupby_columns_1 = ['mkt_carrier', 'origin_airport_id']
columns = ['dep_delay', 'arr_delay'] # columns on which to get aggregate data
flights = flights.groupby(
    groupby_columns_1,group_keys=False).apply( # for a given month
        lambda x: aggregate(x,columns,'carrier_origin_month').groupby(
    groupby_columns_1,group_keys=False).apply( # for a given WEEK OF THE YEAR
        lambda x: aggregate(x,columns,'carrier_origin_week')).groupby(
    groupby_columns_1,group_keys=False).apply( # for a given DAY OF YEAR
        lambda x: aggregate(x,columns,'carrier_origin_date')))


In [91]:
# Divide the flight into short, medium, and long haul flights based on air-time
# SH 2022-10-28 8:55: 'air_time' not available in test data, so use crs_elapsed_time instead.
length=[]

for i in flights['crs_elapsed_time']:
    if i < (180): # less than 3 hours
        length.append('short')
    elif (i >= (180)) and (i <= (360)): #between 3 and 6 hours
        length.append('medium')
    else: length.append('long') # more than 6 hours

In [92]:
    # Replace day numbers with day names
flights['day_of_week'].replace({ 
        0: 'Monday', 
        1: 'Tuesday', 
        2: 'Wednesday', 
        3: 'Thursday', 
        4: 'Friday', 
        5: 'Saturday', 
        6: 'Sunday'}, inplace=True)

In [93]:
    # GROUP STATES INTO REGIONS #Anastasia's code (test is the flights table object name)
regions = {
    'WA': 'West',
    'OR': 'West',
    'ID': 'West',
    'MT': 'West',
    'WY': 'West',
    'CA': 'West',
    'NV': 'West',
    'UT': 'West',
    'AZ': 'West',
    'CO': 'West',
    'NM': 'West',
    'HI': 'West',
    'AK': 'West',
    
    'ND': 'Midwest',
    'SD': 'Midwest',
    'NE': 'Midwest',
    'KS': 'Midwest',
    'MN': 'Midwest',
    'IA': 'Midwest',
    'MO': 'Midwest',
    'WI': 'Midwest',
    'IL': 'Midwest',
    'IN': 'Midwest',
    'MI': 'Midwest',
    'OH': 'Midwest',

    'OK': 'South',
    'TX': 'South',
    'AR': 'South',
    'LA': 'South',
    'MS': 'South',
    'AL': 'South',
    'TN': 'South',
    'KY': 'South',
    'WV': 'South',
    'DC': 'South',
    'VA': 'South',
    'DE': 'South',
    'MD': 'South',
    'NC': 'South',
    'SC': 'South',
    'GA': 'South',
    'FL': 'South',

    'PA': 'Northeast',
    'NY': 'Northeast',
    'VT': 'Northeast',
    'ME': 'Northeast',
    'NH': 'Northeast',
    'MA': 'Northeast',
    'CT': 'Northeast',
    'RI': 'Northeast',
    'NJ': 'Northeast'
    }
# Split City and State 
flights[['origin_city', 'origin_state']] = flights['origin_city_name'].str.split(", ",expand=True,) 
flights[['dest_city', 'dest_state']] = flights['dest_city_name'].str.split(", ",expand=True,) 

flights=flights.replace({"origin_state": regions})
flights=flights.replace({"dest_state": regions})
flights = flights.rename(columns={'origin_state': 'origin_region', 'dest_state': 'dest_region'})

In [94]:
    # CONVERT crs_dep_time TO DATETIME OBJECT
column = 'crs_dep_time'
flights = time_columns(flights,column,format='%H%M',fillna=1,dropna=True)

    # To flights table, add columns with the  features for date and forecasting date
train = date_forecast_columns(flights,date_column='fl_date',format='%Y-%m-%d')
flights.columns

Index(['mkt_carrier', 'mkt_carrier_fl_num', 'fl_date', 'origin_airport_id',
       'dest_airport_id', 'crs_dep_time', 'dep_time', 'dep_delay',
       'crs_arr_time', 'arr_time', 'arr_delay', 'crs_elapsed_time', 'air_time',
       'distance', 'origin_city_name', 'dest_city_name', 'haul_length',
       'crs_arr_hrs', 'crs_dep_hrs', 'dep_hrs_ctg', 'arr_hrs_ctg',
       'crs_dep_time_dt', 'crs_dep_time_hour', 'fl_date_dt', 'year', 'month',
       'day_of_week', 'day_of_year', 'fl_date_year_month',
       'fl_date_Monday_of_week', 'fl_date_week_number',
       'fl_date_t-1_week_week_number', 'fl_date_t-1_week_date',
       'fl_date_t-1_year_year', 'fl_date_t-1_year_month',
       'fl_date_t-1_year_day', 'mean_dep_delay_carrier_origin_month',
       'mean_arr_delay_carrier_origin_month',
       'mean_dep_delay_carrier_origin_week',
       'mean_arr_delay_carrier_origin_week',
       'mean_dep_delay_carrier_origin_date',
       'mean_arr_delay_carrier_origin_date', 'origin_city', 'origin_regi

In [95]:
# TRAIN DATA ONLY - FORECASTING DATA COLUMNS: ['dep_delay', 'arr_delay']
# columns for self-join using forecasting columns
columns_list = [
    'mean_dep_delay_carrier_origin_date',
    'mean_arr_delay_carrier_origin_date'
]
# Mean delay for the 7 days ago
groupby_tm1_week_date = groupby_columns_1 + ['fl_date_t-1_week_date'] 
# Mean delay for the 1 week before
groupby_tm1_week_week_number = groupby_columns_1 + ['fl_date_t-1_week_week_number']
# Mean delay for the same week 1 year ago
groupby_tm1_year_week = groupby_columns_1 + ['year','fl_date_week_number'] # SH 2022-10-27 19:11 updated
# Mean delay for the same MONTH 1 year ago; 
groupby_tm1_year_month = groupby_columns_1 + ['month'] # SH 2022-10-27 19:11 updated

    # ADD FORECASTING COLUMNS WITH SELF-JOIN
flights = flights.merge( 
                flights.filter(items=columns_list + groupby_tm1_week_date).groupby(groupby_tm1_week_date).mean(), # Mean delay for the 7 days ago
                how='left',
                # indicator='merge_tm1_week_date',
                left_on=groupby_columns_1+['fl_date_dt'],
                right_index=True,
                suffixes=[None,'_t-1_week'],
                copy=False
        ).merge( 
                flights.filter(items=columns_list + groupby_tm1_week_week_number).groupby(groupby_tm1_week_week_number).mean(), # Mean delay for the 1 week before # YES
                how='left',
                # indicator='merge_tm1_week_week_number',
                left_on=groupby_columns_1+['fl_date_week_number'],
                right_index=True,
                suffixes=[None,'_t-1_week_week_number'],
                copy=False
        ).merge(
                flights.filter(items=columns_list+ groupby_tm1_year_week).groupby(groupby_tm1_year_week).mean(), # Mean delay for the same week 1 year ago
                how='left',
                # indicator='merge_tm1_year_week',
                right_index=True, 
                left_on=groupby_columns_1+['fl_date_t-1_year_year','fl_date_week_number'],
                suffixes=[None,'t-1_year_week'],
                copy=False
        ).merge( 
                flights.filter(items=columns_list+groupby_tm1_year_month).groupby(groupby_tm1_year_month).mean(),# Mean delay for the same MONTH 1 year ago; 
                how='left',
                # indicator='merge_tm1_year_month',
                left_on=groupby_columns_1+['fl_date_t-1_year_month'],
                right_index=True,
                suffixes=[None,'_t-1_year_month'],
                copy=False
)
flights.shape

(280344, 54)

# Join Passengers Info

In [96]:
passengers = load_csv2(r'C:\Users\silvh\OneDrive\lighthouse\projects\mid-term-project-I\data',
'passengers_2022-10-27_1727.csv',column1_as_index=True)
passengers.shape

(515663, 14)

In [97]:
# MERGE TRAINING FLIGHTS TABLE WITH PASSENGERS TABLE

    # Calculate average per departure
col_for_averaging = [
    'payload',
    'seats',
    'passengers',
    'freight',
    'mail',
]
    # Calculate average per departure
for column in col_for_averaging:
    passengers[str('mean_'+column+'_per_departure')] = passengers[column] / passengers['departures_performed']

    # calculate mean empty seats
passengers['mean_empty_seats_per_departure'] = passengers['mean_seats_per_departure'] - passengers['mean_passengers_per_departure']

    # Drop columns whose data were put into the new columns as 'mean_.*'
passengers.drop(columns=col_for_averaging, inplace=True)

    # Prepare for the merge!
left_join_columns = ['mkt_carrier','origin_airport_id', 'dest_airport_id', 
    'fl_date_t-1_year_year', 'month']
right_join_columns = ['carrier','origin_airport_id', 'dest_airport_id', 
    'year', 'month'] # SH 2022-10-27 20:42 changed 'unique_carrier' to 'carrier'

    # Join flights and passengers table
flights = flights.merge(
    passengers.groupby(right_join_columns).mean(), # SH 2022-10-29 12:02 update to calculate aggregate mean
    how='left',left_on=left_join_columns, 
    right_on=right_join_columns)

print(flights.columns)

# # check that the new columns have values and not all NaNs.
# flights.sort_values('mean_seats_per_departure')

C:\Users\silvh\AppData\Local\Temp\ipykernel_23412\2190203044.py:29: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  passengers.groupby(right_join_columns).mean(), # SH 2022-10-29 12:02 update to calculate aggregate mean


Index(['mkt_carrier', 'mkt_carrier_fl_num', 'fl_date', 'origin_airport_id',
       'dest_airport_id', 'crs_dep_time', 'dep_time', 'dep_delay',
       'crs_arr_time', 'arr_time', 'arr_delay', 'crs_elapsed_time', 'air_time',
       'distance_x', 'origin_city_name', 'dest_city_name', 'haul_length',
       'crs_arr_hrs', 'crs_dep_hrs', 'dep_hrs_ctg', 'arr_hrs_ctg',
       'crs_dep_time_dt', 'crs_dep_time_hour', 'fl_date_dt', 'year', 'month',
       'day_of_week', 'day_of_year', 'fl_date_year_month',
       'fl_date_Monday_of_week', 'fl_date_week_number',
       'fl_date_t-1_week_week_number', 'fl_date_t-1_week_date',
       'fl_date_t-1_year_year', 'fl_date_t-1_year_month',
       'fl_date_t-1_year_day', 'mean_dep_delay_carrier_origin_month',
       'mean_arr_delay_carrier_origin_month',
       'mean_dep_delay_carrier_origin_week',
       'mean_arr_delay_carrier_origin_week',
       'mean_dep_delay_carrier_origin_date',
       'mean_arr_delay_carrier_origin_date', 'origin_city', 'origin_re

In [98]:
# drop extra columns from the merge
flights.drop(columns='distance_y', inplace = True)
sorted(flights.columns)

['air_time',
 'airline_id',
 'arr_delay',
 'arr_hrs_ctg',
 'arr_time',
 'crs_arr_hrs',
 'crs_arr_time',
 'crs_dep_hrs',
 'crs_dep_time',
 'crs_dep_time_dt',
 'crs_dep_time_hour',
 'crs_elapsed_time',
 'day_of_week',
 'day_of_year',
 'dep_delay',
 'dep_hrs_ctg',
 'dep_time',
 'departures_performed',
 'dest_airport_id',
 'dest_city',
 'dest_city_name',
 'dest_region',
 'distance_x',
 'fl_date',
 'fl_date_Monday_of_week',
 'fl_date_dt',
 'fl_date_t-1_week_date',
 'fl_date_t-1_week_week_number',
 'fl_date_t-1_year_day',
 'fl_date_t-1_year_month',
 'fl_date_t-1_year_year',
 'fl_date_week_number',
 'fl_date_year_month',
 'haul_length',
 'mean_arr_delay_carrier_origin_date',
 'mean_arr_delay_carrier_origin_date_t-1_week',
 'mean_arr_delay_carrier_origin_date_t-1_week_week_number',
 'mean_arr_delay_carrier_origin_date_t-1_year_month',
 'mean_arr_delay_carrier_origin_datet-1_year_week',
 'mean_arr_delay_carrier_origin_month',
 'mean_arr_delay_carrier_origin_week',
 'mean_dep_delay_carrier_origi

In [99]:
explore(flights)

Unique IDs: 11. # of rows: 280344. Match: False



,Total,Percent
mean_dep_delay_carrier_origin_date_t-1_week,280344,1.000000
mean_arr_delay_carrier_origin_date_t-1_week,280344,1.000000
mean_empty_seats_per_departure,176299,0.628867
airline_id,176299,0.628867
mean_mail_per_departure,176299,0.628867
...,...,...
fl_date_year_month,0,0.000000
fl_date_Monday_of_week,0,0.000000
fl_date_week_number,0,0.000000
mkt_carrier_fl_num,0,0.000000


In [100]:
# FLIGHTS TABLE TRAINING DATA: Fill mean flight historical/forecasting delay data. 
dict = {
    'mean_dep_delay_carrier_origin_datet-1_year_week': 'mean_dep_delay_carrier_origin_week', 
    'mean_arr_delay_carrier_origin_datet-1_year_week': 'mean_arr_delay_carrier_origin_week', 
    'mean_dep_delay_carrier_origin_date_t-1_year_month': 'mean_dep_delay_carrier_origin_month', 
    'mean_arr_delay_carrier_origin_date_t-1_year_month': 'mean_arr_delay_carrier_origin_month' 
}
fill_missing(flights,dict,fill_w_mean=False) # Call the function

,mkt_carrier,mkt_carrier_fl_num,fl_date,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,dep_delay,crs_arr_time,arr_time,...,mean_dep_delay_carrier_origin_date_t-1_year_month,mean_arr_delay_carrier_origin_date_t-1_year_month,departures_performed,airline_id,mean_payload_per_departure,mean_seats_per_departure,mean_passengers_per_departure,mean_freight_per_departure,mean_mail_per_departure,mean_empty_seats_per_departure
0,AA,324,2019-01-06,13930,13198,1340,1352.0,12.0,1507,1520.0,...,20.251264,16.987945,31.50,19805.0,37787.903226,150.0,86.661290,38.669355,1.927419,63.338710
1,DL,5402,2018-01-07,10397,14098,956,954.0,-2.0,1138,1131.0,...,6.404275,-1.417302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,WN,6435,2018-01-07,10800,14831,1755,1832.0,37.0,1905,1930.0,...,10.737276,6.389271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,UA,467,2019-01-05,11042,11292,1534,1525.0,-9.0,1703,1649.0,...,8.988593,0.236190,15.50,19977.0,43109.203704,172.5,161.773148,62.703704,1196.560185,10.726852
4,DL,5790,2019-01-05,14869,14831,1111,1105.0,-6.0,1221,1232.0,...,6.884221,-2.491205,31.75,19790.0,36036.250000,140.5,107.704268,5.930894,19.599593,32.795732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280339,UA,3630,2018-01-02,13930,14122,2110,2116.0,6.0,2338,2349.0,...,21.723602,17.490610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
280340,DL,1973,2018-01-06,13930,10397,828,825.0,-3.0,1130,1127.0,...,9.407787,2.511340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
280341,AA,4643,2018-01-02,10693,11057,1010,1005.0,-5.0,1240,1232.0,...,15.897112,9.309222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
280342,VX,1983,2018-01-04,12892,14771,2145,2137.0,-8.0,2312,2300.0,...,5.362934,6.092664,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
explore(flights).head(30)

Unique IDs: 11. # of rows: 280344. Match: False



,Total,Percent
mean_dep_delay_carrier_origin_date_t-1_week,280344,1.000000
mean_arr_delay_carrier_origin_date_t-1_week,280344,1.000000
mean_empty_seats_per_departure,176299,0.628867
departures_performed,176299,0.628867
mean_mail_per_departure,176299,0.628867
mean_freight_per_departure,176299,0.628867
mean_passengers_per_departure,176299,0.628867
mean_seats_per_departure,176299,0.628867
mean_payload_per_departure,176299,0.628867
airline_id,176299,0.628867


In [107]:
# Save df before removing columns and rows or filling in means
save_csv(flights, 'train_with_nulls_', 
    path=r'C:\Users\silvh\OneDrive\lighthouse\projects\mid-term-project-I\data',
    append_version=True)

File saved:  C:/Users/silvh/OneDrive/lighthouse/projects/mid-term-project-I/data/train_with_nulls_2022-10-29_1417


# Remove features and fill missing values

In [110]:
# REMOVE NULLS
    # REMOVE FEATURES WITH NULL VALUES ABOVE THRESHOLD
threshold = 0.99 
drop_features(flights,threshold=threshold)
explore(flights).index

Threshold of percentage values for dropping columns: 0.99
Columns dropped: ['mean_dep_delay_carrier_origin_date_t-1_week', 'mean_arr_delay_carrier_origin_date_t-1_week']
Unique IDs: 11. # of rows: 280344. Match: False



Index(['mean_empty_seats_per_departure', 'mean_seats_per_departure',
       'departures_performed', 'mean_payload_per_departure', 'airline_id',
       'mean_passengers_per_departure', 'mean_freight_per_departure',
       'mean_mail_per_departure',
       'mean_arr_delay_carrier_origin_date_t-1_week_week_number',
       'mean_dep_delay_carrier_origin_date_t-1_week_week_number', 'arr_delay',
       'air_time', 'dep_delay', 'arr_time', 'dep_time',
       'mean_dep_delay_carrier_origin_date',
       'mean_arr_delay_carrier_origin_week', 'fl_date_t-1_year_day',
       'mean_dep_delay_carrier_origin_week',
       'mean_arr_delay_carrier_origin_month',
       'mean_dep_delay_carrier_origin_month', 'origin_city',
       'mean_arr_delay_carrier_origin_date',
       'mean_arr_delay_carrier_origin_datet-1_year_week', 'origin_region',
       'dest_city', 'dest_region',
       'mean_dep_delay_carrier_origin_datet-1_year_week',
       'fl_date_t-1_year_year',
       'mean_dep_delay_carrier_origin_da

In [114]:
# Drop rows with missing values from the passengers table
columns_to_check_nulls = [
    'departures_performed'
    ]

flights2 = flights.dropna(subset=columns_to_check_nulls)
print(flights.shape)
print(flights2.shape)

(280344, 60)
(104045, 60)


In [115]:
explore(flights2)

Unique IDs: 10. # of rows: 104045. Match: False



,Total,Percent
mean_arr_delay_carrier_origin_date_t-1_week_week_number,15458,0.148570
mean_dep_delay_carrier_origin_date_t-1_week_week_number,15458,0.148570
air_time,692,0.006651
arr_delay,692,0.006651
arr_time,562,0.005402
dep_time,516,0.004959
dep_delay,516,0.004959
mean_dep_delay_carrier_origin_week,0,0.000000
mean_arr_delay_carrier_origin_week,0,0.000000
mean_arr_delay_carrier_origin_month,0,0.000000


In [116]:
explore(flights2).index

Unique IDs: 10. # of rows: 104045. Match: False



Index(['mean_arr_delay_carrier_origin_date_t-1_week_week_number',
       'mean_dep_delay_carrier_origin_date_t-1_week_week_number', 'air_time',
       'arr_delay', 'arr_time', 'dep_time', 'dep_delay',
       'mean_dep_delay_carrier_origin_week',
       'mean_arr_delay_carrier_origin_week',
       'mean_arr_delay_carrier_origin_month',
       'mean_dep_delay_carrier_origin_month',
       'mean_dep_delay_carrier_origin_date', 'fl_date_t-1_year_day',
       'mean_arr_delay_carrier_origin_date', 'origin_city', 'origin_region',
       'dest_city', 'dest_region', 'mkt_carrier',
       'mean_arr_delay_carrier_origin_datet-1_year_week',
       'mean_dep_delay_carrier_origin_datet-1_year_week',
       'fl_date_t-1_year_year',
       'mean_dep_delay_carrier_origin_date_t-1_year_month',
       'mean_arr_delay_carrier_origin_date_t-1_year_month',
       'departures_performed', 'airline_id', 'mean_payload_per_departure',
       'mean_seats_per_departure', 'mean_passengers_per_departure',
       'me

In [117]:
# Fill missing values with the mean based on groupby
groupby_columns = ['mkt_carrier', 'origin_airport_id']
columns_to_fill = ['mean_arr_delay_carrier_origin_date_t-1_week_week_number',
       'mean_dep_delay_carrier_origin_date_t-1_week_week_number', 'air_time',
       'arr_delay', 'arr_time', 'dep_time', 'dep_delay']
# columns to fill
flights2 = flights2.groupby(groupby_columns,group_keys=False).apply( 
        lambda x: fill_with_mean(x,columns_to_fill))
explore(flights2)

Unique IDs: 10. # of rows: 104045. Match: False



,Total,Percent
mkt_carrier,0,0.0
mkt_carrier_fl_num,0,0.0
fl_date_t-1_week_date,0,0.0
fl_date_t-1_year_year,0,0.0
fl_date_t-1_year_month,0,0.0
fl_date_t-1_year_day,0,0.0
mean_dep_delay_carrier_origin_month,0,0.0
mean_arr_delay_carrier_origin_month,0,0.0
mean_dep_delay_carrier_origin_week,0,0.0
mean_arr_delay_carrier_origin_week,0,0.0


In [119]:
# Save flights data with missing values removed.
save_csv(flights2, 'train_without_nulls_', 
    path=r'C:\Users\silvh\OneDrive\lighthouse\projects\mid-term-project-I\data',
    append_version=True)

File saved:  C:/Users/silvh/OneDrive/lighthouse/projects/mid-term-project-I/data/train_without_nulls_2022-10-29_1500


# Make dummy variables

In [146]:
# FLIGHTS TABLE - DUMMY VARIABLES
dummy = [
    'origin_region',
'dest_region',
'dep_hrs_ctg', 'arr_hrs_ctg',
'day_of_week',
'haul_length',
]
flights3 = pd.get_dummies(flights2,columns=dummy)

# Rename distance_x to distance
flights3 = flights3.rename(columns={'distance_x': 'distance'})
flights3.columns

# Load test data

In [135]:
test = load_csv2(r'C:\Users\silvh\OneDrive\lighthouse\projects\mid-term-project-I\data',
    'test_prepared_2022-10-29_1135.csv', column1_as_index=True)

In [148]:
# columns to keep
columns_for_ID =[
'fl_date',
 'mkt_carrier',
 'mkt_carrier_fl_num','origin_airport_id', 'dest_airport_id']
outcomes = ['arr_delay','dep_delay']
# See the differences in columns between the train and test data sets
column_disparity =  list(set(flights3.columns) - set(test.columns) - set(columns_for_ID) - set(outcomes))

# Drop the extra columns
flights3.drop(columns=column_disparity,inplace=True)
list(set(flights3.columns) - set(test.columns))

['dest_airport_id', 'dep_delay', 'arr_delay', 'origin_airport_id']

In [149]:
list(set(test.columns) - set(flights3.columns))

[]

# Remove outliers for arr_delay

In [150]:
# Remove outliers for arr_delay
# Upper limit and lower limit using 3 standard deviations.
upper = flights3['arr_delay'].mean() + 3 * flights3['arr_delay'].std()
lower = flights3['arr_delay'].mean() - 3 * flights3['arr_delay'].std()

# Filter out the outliers
flights3 = flights3[(flights3['arr_delay'] < upper) & (flights3['arr_delay'] > lower)]

In [151]:
# Save flights data with outliers and unneeded columns removed.
save_csv(flights2, 'train_for_model_', 
    path=r'C:\Users\silvh\OneDrive\lighthouse\projects\mid-term-project-I\data',
    append_version=True)

File saved:  C:/Users/silvh/OneDrive/lighthouse/projects/mid-term-project-I/data/train_for_model_2022-10-29_1538


In [103]:
columns_for_ID = [
'fl_date',
 'mkt_carrier',
 'mkt_carrier_fl_num','origin_airport_id', 'dest_airport_id']

df.set_index(columns_for_ID)

NameError: name 'df' is not defined

# *End of Page*